Install two libraries for the edited code

https://shap.readthedocs.io/en/latest/index.html

https://mapie.readthedocs.io/en/latest/quick_start.html

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import TimeSeriesSplit
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

from mapie.metrics import regression_coverage_score, regression_mean_width_score
from mapie.regression import MapieRegressor

In [ ]:
best_params = {'''put your best hyperparameters here'''}

## Predict median - This part is based on Jiahui's code. We only need 'pred_median'

median_pipeline = Pipeline([('scaler', MinMaxScaler()), ('median_model', 
                                                         ''' put your regressor here, 
                                                         e.g xg.XGBRegressor ''')
median_pipeline.fit(x_train, y_train)
pred_median = median_pipeline.predict(x_test)


# Evaluate model performance at record level

print('RMSE = ' + str(mean_squared_error(y_test, pred_median, squared=False)))

# Fit estimated prediction intervals

mapie = MapieRegressor(
    median_pipeline, method="naive", cv=TimeSeriesSplit(), agg_function=None, n_jobs=-1
)

mapie.fit(x_train, y_train)
pred_median, y_pis = mapie.predict(x_test, alpha=0.05)
coverage = regression_coverage_score(y_test, y_pis[:, 0, 0], y_pis[:, 1, 0])
width = regression_mean_width_score(y_pis[:, 0, 0], y_pis[:, 1, 0])

# Plot estimated prediction intervals
                            
fig = plt.figure(figsize=(15, 5))
ax = fig.add_subplot(1, 1, 1)
plt.ylabel("vent_duration (h)")
ax.plot(y_test, lw=2, label="Actual", c="C0")
ax.plot(y_test.index, pred_median, lw=2, c="C1", label="Predicted")
ax.set_ylim(bottom=0)
ax.fill_between(
    y_test.index,
    y_pis[:, 0, 0],
    y_pis[:, 1, 0],
    color="C1",
    alpha=0.2,
    label="Prediction Interval",
)
plt.title("Our model: ventilation duration (hrs) - actual vs predicted")
ax.legend()
plt.show()

# For Tree-based models: Feature scores and SHAP values

# Note: the feature scores here are based on xgnoost package. Ordinary package scikit has different method names.                             
print("## Feature scores ##")
print()
#‘weight’: the number of times a feature is used to split the data across all trees.
#‘gain’: the average gain across all splits the feature is used in.
feature_importance = median_pipeline[1].get_booster().get_score(importance_type='gain')
feature = median_pipeline[1].get_booster().feature_types

keys = list(feature_importance.keys())
values = list(feature_importance.values())

sorted_feature_importance = []
for i in range(len(feature_importance)):
    sorted_feature_importance.append((keys[i],values[i]))

sorted_feature_importance = sorted(sorted_feature_importance, reverse=True, key=lambda feature: feature[1])
for i in sorted_feature_importance:
    print(i[0],":", i[1])
print()

# For ordinary package scikit tree-based models use method names that look like this:
'''    
for i in range(len(median_pipeline.feature_names_in_)):
    print(median_pipeline.feature_names_in_[i], ":" , median_pipeline[1].feature_importances_[i])
'''
                            
import shap
print("## Shapley Additive Explanations - SHAP value ##")
print()
explainer = shap.Explainer(median_pipeline[1])
shap_values = explainer(x_test)
shap.plots.bar(shap_values, max_display=12) # default is max_display=12
                            
shap.summary_plot(shap_values, x_test)